In [1]:
from netCDF4 import Dataset
import numpy as np
import pandas as pd

In [2]:
path='‪D:/22monidata/wrfout_d02_2017-01-02_00_00_00'
raw_data=Dataset(path)

In [3]:
#(25,85,93,90)(nt,nl,ny,nx)第一维标签

In [4]:
nx = pd.Series(np.arange(1,91)) #nx东西格点数

In [5]:
#(25,85,93,90)(nt,nl,ny,nx)第一维标签

In [6]:
ny = pd.Series(np.arange(1,94)) #ny南北格点数

In [7]:
#(25,85,93,90)(nt,nl,ny,nx)第一维标签

In [8]:
nl = pd.Series(np.arange(1,86))

In [9]:
#(25,85,93,90)(nt,nl,ny,nx)第一维标签

In [10]:
nt = pd.Series(np.arange(1,26))

In [11]:
#展开标签

In [12]:
import itertools
# index的笛卡尔乘积。注意：高维在前，低维在后
prod = itertools.product(nx,ny,nl,nt)
# 转换为DataFrame
prod = pd.DataFrame([x for x in prod])
prod.columns = ['nx','ny', 'nl', 'nt']
prod.T

,0,1,2,3,4,5,6,7,8,9,...,17786240,17786241,17786242,17786243,17786244,17786245,17786246,17786247,17786248,17786249
nx,1,1,1,1,1,1,1,1,1,1,...,90,90,90,90,90,90,90,90,90,90
ny,1,1,1,1,1,1,1,1,1,1,...,93,93,93,93,93,93,93,93,93,93
nl,1,1,1,1,1,1,1,1,1,1,...,85,85,85,85,85,85,85,85,85,85
nt,1,2,3,4,5,6,7,8,9,10,...,16,17,18,19,20,21,22,23,24,25


In [13]:
# 所有使用到的变量名
used_features = [
    'Times',   # 时间
    'XLAT',    # 维度
    'XLONG',   # 经度
    'U',       # 东西风力
    'V',       # 南北风力
    'PH',      # perturbation geopotential，扰动位势
    'PHB',     # base-state geopotential，基态位势
    'T',       # perturbation potential temperature，扰动潜在温度
    'P',       # perturbation pressure，扰动压强
    'PB',      # base state pressure，基态压强
    'HGT',     # Terrain height，海拔高度
    'QVAPOR',  # 水汽混合比
]

In [14]:
raw_data.variables['T'][:]    # 访问变量数据的方法，返回numpy掩码数组
np.array(raw_data.variables['T'][:])     # 可以直接转成numpy数组
# 利用‘P’和‘PB’计算气压值pressure
pressure = (raw_data.variables['P'][:] + raw_data.variables['PB'][:]) / 100
pressure.shape

(25, 85, 93, 90)

In [15]:
# 利用‘PH’和‘PHB’计算高度值height
PH, PHB = raw_data.variables['PH'][:], raw_data.variables['PHB'][:]
PHH = 0.5 * (PH[:, :85, :, :] + PH[:, 1:86, :, :])
PHBH = 0.5 * (PHB[:, :85, :, :] + PHB[:, 1:86, :, :])
GEOPT = PHH + PHBH
height = GEOPT / 9.81   # 求高度值
height.shape

(25, 85, 93, 90)

In [16]:
# 利用‘T’和‘pressure’计算温度值T
T = raw_data.variables['T'][:]
Tem = (T + 300.) / (1000. / pressure) **0.286
Tem.shape

(25, 85, 93, 90)

In [17]:
# 利用‘QVAPOR’计算水气压值
QVAPOR = raw_data.variables['QVAPOR'][:]  # 水汽混合比
e = QVAPOR * pressure / (0.622 + QVAPOR) 
QVAPOR.shape,e.shape

((25, 85, 93, 90), (25, 85, 93, 90))

In [18]:
XLONG = raw_data.variables['XLONG'][:]    # 读取经度值
HGT = raw_data.variables['HGT'][:]    # 读取海拔高度
XLAT = raw_data.variables['XLAT'][:]    # 读取纬度值
XLONG.shape,XLAT.shape,HGT.shape

((25, 93, 90), (25, 93, 90), (25, 93, 90))

In [19]:
xlong, xlat,hgt = np.zeros((25,93,90,85)), np.zeros((25,93,90,85)), np.zeros((25,93,90,85))
#转化为四维数组
for i in range(85):
    xlong[:,:,:,i]=XLONG[:,:,:]
    xlat[:,:,:,i]=XLAT[:,:,:]
    hgt[:,:,:,i]=HGT[:,:,:]
xlong.shape,xlat.shape,hgt.shape

((25, 93, 90, 85), (25, 93, 90, 85), (25, 93, 90, 85))

In [20]:
# 根据Fortran的代码转化数据维度  (nx东西格点数, ny南北格点数, nl垂直层数, nt时次数)
e = e.transpose((3, 2, 1, 0))
Tem = Tem.transpose((3, 2, 1, 0))
QVAPOR = QVAPOR.transpose((3, 2, 1, 0))
pressure = pressure.transpose((3, 2, 1, 0))
height = height.transpose((3, 2, 1, 0))
xlong = xlong.transpose((2, 1, 3,0))
hgt = hgt.transpose((2, 1, 3,0))
xlat = xlat.transpose((2, 1, 3,0))
z=height-hgt
[e.shape, Tem.shape, pressure.shape, height.shape, hgt.shape, xlat.shape ,xlong.shape,z.shape]

[(90, 93, 85, 25),
 (90, 93, 85, 25),
 (90, 93, 85, 25),
 (90, 93, 85, 25),
 (90, 93, 85, 25),
 (90, 93, 85, 25),
 (90, 93, 85, 25),
 (90, 93, 85, 25)]

In [21]:
# 计算折射率，NN是原始折射率，MM是修正折射率
re = 6371.
NN = (3.73*10**5*e)/(Tem**2)+(77.6*pressure)/(Tem)
MM = (3.73*10**5*e)/(Tem**2)+(77.6*pressure)/(Tem)+((height-hgt)/(re*1000.)*10**6)
NN.shape, MM.shape

((90, 93, 85, 25), (90, 93, 85, 25))

In [22]:
a=xlong.flatten()
a = pd.Series(a)
a.name = 'xlong'

b=xlat.flatten()
b = pd.Series(b)
b.name = 'xlat'

c=hgt.flatten()
c = pd.Series(c)
c.name = 'hgt'

o=height.flatten()
o=pd.Series(o)
o.name='height'

d=z.flatten()
d = pd.Series(d)
d.name = 'z'

f=e.flatten()
f = pd.Series(f)
f.name = 'e'

g=pressure.flatten()
g = pd.Series(g)
g.name = 'pressure'

h=Tem.flatten()
h = pd.Series(h)
h.name = 'Tem'

l=QVAPOR.flatten()
l = pd.Series(l)
l.name = 'QVAPOR'

mm=MM.flatten()
mm = pd.Series(mm)
mm.name = 'MM'

nn=NN.flatten()
nn = pd.Series(nn)
nn.name = 'NN'

In [23]:
#最终数据，合并成一个DataFrame
data=pd.concat([prod,a,b,c,o,d,f,g,h,l,mm,nn],axis=1)
data1=pd.DataFrame(data)
data2=data1.iloc[0:197625,:]
data2

,nx,ny,nl,nt,xlong,xlat,hgt,z,e,pressure,Tem,QVAPOR,MM,NN
0,1,1,1,1,104.101074,1.387764,18.679893,3.232424,29.763758,1005.953430,299.462860,0.018965,384.977855,384.470490
1,1,1,1,2,104.101074,1.387764,18.679893,3.240555,29.191788,1006.418579,300.404114,0.018580,381.143834,380.635193
2,1,1,1,3,104.101074,1.387764,18.679893,3.241436,29.299547,1006.550781,300.425964,0.018649,381.586996,381.078217
3,1,1,1,4,104.101074,1.387764,18.679893,3.243279,29.517775,1006.709351,300.602356,0.018789,382.234411,381.725342
4,1,1,1,5,104.101074,1.387764,18.679893,3.245729,29.570362,1005.952942,300.748413,0.018838,382.011956,381.502502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197620,1,93,85,21,101.212708,23.116425,1303.248901,15018.905396,0.000294,103.884392,194.972870,0.000002,2398.735173,41.349300
197621,1,93,85,22,101.212708,23.116425,1303.248901,15011.298950,0.000303,103.802467,194.897842,0.000002,2397.524639,41.332684
197622,1,93,85,23,101.212708,23.116425,1303.248901,15011.410278,0.000309,103.801941,194.784470,0.000002,2397.566020,41.356590
197623,1,93,85,24,101.212708,23.116425,1303.248901,15011.165161,0.000309,103.800735,194.479874,0.000002,2397.591851,41.420895


In [24]:
data2.to_csv("D:/20年海洋/数据/模拟数据/时间层面导出数据/20170102nx1.csv" , encoding = "utf-8", header=True, index=None)